In [4]:
import torch
import torchvision
from torchvision.models.detection import retinanet_resnet50_fpn
from torchvision.models.detection import RetinaNet_ResNet50_FPN_Weights
from torch.utils.data import DataLoader, WeightedRandomSampler
from torchvision.datasets import CocoDetection
import torchvision.transforms as T
import json
from tqdm.auto import tqdm
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
from collections import defaultdict
import os
import numpy as np

#RETINANET for sign conditions

train_json = r'COCO-based_COCO_condition\annotations\train.json'
val_json = r'COCO-based_COCO_condition\annotations\val.json'
train_img_dir = r'COCO-based_COCO_condition\images\train'
val_img_dir = r'COCO-based_COCO_condition\images\val'

# conditions
allowed_conditions = ["Good", "Weathered", "Heavily Damaged"]

# Number of classes: 3 conditions + 1 background = 4
num_classes = 4

# parameters
BATCH_SIZE = 4
NUM_EPOCHS = 15
LEARNING_RATE = 0.0001
NUM_WORKERS = 0  

# Early stopping
PATIENCE = 5

# TensorBoard configuration
EXPERIMENT_NAME = f"retinanet_balanced_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
LOG_DIR = os.path.join('runs', EXPERIMENT_NAME)


class FilteredCocoDetection(CocoDetection):
    def __init__(self, root, annFile, allowed_conditions):
        super().__init__(root, annFile)
        self.annFile = annFile
        self.allowed_conditions = allowed_conditions
        self._filter_annotations()
    
    def _filter_annotations(self):
        """Filter annotations to only include allowed conditions"""
        with open(self.annFile, 'r') as f:
            data = json.load(f)
        
        filtered_anns = []
        for ann in data['annotations']:
            if 'attributes' in ann and 'condition' in ann['attributes']:
                condition = ann['attributes']['condition'][0]
                if condition in self.allowed_conditions:
                    ann['category_id'] = self.allowed_conditions.index(condition) + 1
                    filtered_anns.append(ann)
        
        print(f"Filtered {len(data['annotations'])} -> {len(filtered_anns)} annotations")
        
        data['annotations'] = filtered_anns
        data['categories'] = [
            {'id': i+1, 'name': cond} 
            for i, cond in enumerate(self.allowed_conditions)
        ]
        
        self.coco.dataset = data
        self.coco.createIndex()
    
    def __getitem__(self, idx):
        """Override to return proper format for RetinaNet"""
        img, target = super().__getitem__(idx)
        
        # Convert image to tensor
        img = T.ToTensor()(img)
        
        # Convert target to proper format
        boxes = []
        labels = []
        
        for obj in target:
            if 'bbox' in obj and 'category_id' in obj:
                x, y, w, h = obj['bbox']
                boxes.append([x, y, x + w, y + h])
                labels.append(obj['category_id'])
        
        if len(boxes) == 0:
            boxes = torch.zeros((0, 4), dtype=torch.float32)
            labels = torch.zeros((0,), dtype=torch.int64)
        else:
            boxes = torch.as_tensor(boxes, dtype=torch.float32)
            labels = torch.as_tensor(labels, dtype=torch.int64)
        
        target_dict = {
            'boxes': boxes,
            'labels': labels
        }
        
        return img, target_dict


model = retinanet_resnet50_fpn(weights=RetinaNet_ResNet50_FPN_Weights.DEFAULT)

# Modify classification head for your number of classes
in_features = 256
num_anchors = model.head.classification_head.num_anchors

from torchvision.models.detection.retinanet import RetinaNetClassificationHead
model.head.classification_head = RetinaNetClassificationHead(
    in_channels=in_features,
    num_anchors=num_anchors,
    num_classes=num_classes
)


print("Training set:")
train_dataset = FilteredCocoDetection(
    root=train_img_dir, 
    annFile=train_json, 
    allowed_conditions=allowed_conditions
)

print("\nValidation set:")
val_dataset = FilteredCocoDetection(
    root=val_img_dir, 
    annFile=val_json, 
    allowed_conditions=allowed_conditions
)

print(f"\nDataset sizes:")
print(f"  Train: {len(train_dataset)} images")
print(f"  Val: {len(val_dataset)} images")


# Count instances per class and track each sample's classes
class_instance_count = defaultdict(int)
sample_labels = []

for idx in range(len(train_dataset)):
    _, target = train_dataset[idx]
    labels = target['labels'].tolist()
    
    if labels:
        sample_labels.append(labels)
        for label in labels:
            class_instance_count[label] += 1
    else:
        sample_labels.append([1])  # Default to class 1 if no labels

print("\nOriginal class distribution (instances):")
for cls in sorted(class_instance_count.keys()):
    count = class_instance_count[cls]
    pct = 100 * count / sum(class_instance_count.values())
    print(f"  {allowed_conditions[cls-1]}: {count} ({pct:.1f}%)")

# Calculate class weights (inverse frequency)
total_instances = sum(class_instance_count.values())
class_weights = {}
for cls, count in class_instance_count.items():
    # Weight is inversely proportional to frequency
    # Add smoothing to avoid extreme weights
    class_weights[cls] = total_instances / (count * len(class_instance_count))

print("\nClass weights (for sampling):")
for cls in sorted(class_weights.keys()):
    print(f"  {allowed_conditions[cls-1]}: {class_weights[cls]:.2f}")

# Assign weight to each sample based on rarest class present
sample_weights = []
for labels in sample_labels:
    # Weight by the rarest class in the image
    weights = [class_weights[label] for label in labels]
    sample_weights.append(max(weights))

# Create weighted random sampler
sampler = WeightedRandomSampler(
    weights=sample_weights,
    num_samples=len(sample_weights),
    replacement=True
)

print(f"\nWeighted sampler created with {len(sample_weights)} samples")

def collate_fn(batch):
    return tuple(zip(*batch))

train_loader = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    sampler=sampler,  
    num_workers=NUM_WORKERS, 
    collate_fn=collate_fn
)

val_loader = DataLoader(
    val_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False, 
    num_workers=NUM_WORKERS, 
    collate_fn=collate_fn
)

print(f"\nSteps per epoch:")
print(f"  Train: {len(train_loader)} steps")
print(f"  Val: {len(val_loader)} steps")


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\nUsing device: {device}")

model.to(device)

# Optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.AdamW(params, lr=LEARNING_RATE, weight_decay=0.0001)

# Learning rate scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=3
)


writer = SummaryWriter(log_dir=LOG_DIR)

writer.add_text('Hyperparameters', f'''
- Batch Size: {BATCH_SIZE}
- Learning Rate: {LEARNING_RATE}
- Weight Decay: 0.0005
- Optimizer: AdamW
- Scheduler: ReduceLROnPlateau (patience=5, factor=0.5)
- Number of Classes: {num_classes}
- Conditions: {", ".join(allowed_conditions)}
- Train Images: {len(train_dataset)}
- Val Images: {len(val_dataset)}
- **BALANCED SAMPLING ENABLED**
- Class Weights: Good={class_weights[1]:.2f}, Weathered={class_weights[2]:.2f}, Heavily Damaged={class_weights[3]:.2f}
''')


#tensorboard --logdir=runs


print(f"\nStarting training for {NUM_EPOCHS} epochs")


best_val_loss = float('inf')
patience_counter = 0
global_step = 0

for epoch in range(NUM_EPOCHS):

    model.train()
    train_loss = 0
    train_cls_loss = 0
    train_box_loss = 0
    
    # Track which classes appear in batches
    epoch_class_counts = defaultdict(int)
    
    pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{NUM_EPOCHS} [Train]')
    for batch_idx, (images, targets) in enumerate(pbar):
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        # Track class distribution in this batch
        for target in targets:
            for label in target['labels']:
                epoch_class_counts[label.item()] += 1
        
        # Skip empty batches
        if any(len(t['boxes']) == 0 for t in targets):
            continue
        
        # Forward pass
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        
        cls_loss = loss_dict.get('classification', torch.tensor(0.0))
        box_loss = loss_dict.get('bbox_regression', torch.tensor(0.0))
        
        # Backward pass
        optimizer.zero_grad()
        losses.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        train_loss += losses.item()
        train_cls_loss += cls_loss.item()
        train_box_loss += box_loss.item()
        
        # Log to TensorBoard
        writer.add_scalar('Train/Batch_Loss', losses.item(), global_step)
        writer.add_scalar('Train/Batch_Classification_Loss', cls_loss.item(), global_step)
        writer.add_scalar('Train/Batch_BBox_Loss', box_loss.item(), global_step)
        
        global_step += 1
        pbar.set_postfix({'loss': f'{losses.item():.4f}'})
    
    train_loss = train_loss / len(train_loader)
    train_cls_loss = train_cls_loss / len(train_loader)
    train_box_loss = train_box_loss / len(train_loader)
    
    # Log class distribution seen during training
    print(f"\nClasses seen in epoch {epoch+1}:")
    total_seen = sum(epoch_class_counts.values())
    for cls in sorted(epoch_class_counts.keys()):
        count = epoch_class_counts[cls]
        pct = 100 * count / total_seen if total_seen > 0 else 0
        print(f"  {allowed_conditions[cls-1]}: {count} ({pct:.1f}%)")
        writer.add_scalar(f'Training_Distribution/{allowed_conditions[cls-1]}', count, epoch)
    
   
    model.train()
    val_loss = 0
    val_cls_loss = 0
    val_box_loss = 0
    
    with torch.no_grad():
        pbar = tqdm(val_loader, desc=f'Epoch {epoch+1}/{NUM_EPOCHS} [Val]  ')
        for images, targets in pbar:
            images = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            
            if any(len(t['boxes']) == 0 for t in targets):
                continue
            
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())
            
            cls_loss = loss_dict.get('classification', torch.tensor(0.0))
            box_loss = loss_dict.get('bbox_regression', torch.tensor(0.0))
            
            val_loss += losses.item()
            val_cls_loss += cls_loss.item()
            val_box_loss += box_loss.item()
            
            pbar.set_postfix({'loss': f'{losses.item():.4f}'})
    
    val_loss = val_loss / len(val_loader)
    val_cls_loss = val_cls_loss / len(val_loader)
    val_box_loss = val_box_loss / len(val_loader)
    
    # Update scheduler
    scheduler.step(val_loss)
    current_lr = optimizer.param_groups[0]['lr']
    

    
    writer.add_scalars('Loss/Total', {'train': train_loss, 'val': val_loss}, epoch)
    writer.add_scalars('Loss/Classification', {'train': train_cls_loss, 'val': val_cls_loss}, epoch)
    writer.add_scalars('Loss/BBox_Regression', {'train': train_box_loss, 'val': val_box_loss}, epoch)
    
    overfit_ratio = train_loss / val_loss if val_loss > 0 else 1.0
    writer.add_scalar('Metrics/Overfit_Ratio', overfit_ratio, epoch)
    writer.add_scalar('Hyperparameters/Learning_Rate', current_lr, epoch)
    
   
    print(f'\nEpoch {epoch+1}/{NUM_EPOCHS}:')
    print(f'  Train Loss: {train_loss:.4f} (cls: {train_cls_loss:.4f}, bbox: {train_box_loss:.4f})')
    print(f'  Val Loss:   {val_loss:.4f} (cls: {val_cls_loss:.4f}, bbox: {val_box_loss:.4f})')
    print(f'  Overfit Ratio: {overfit_ratio:.3f}')
    print(f'  LR: {current_lr:.6f}')
    
    
    if val_loss < best_val_loss:
        improvement = best_val_loss - val_loss
        best_val_loss = val_loss
        patience_counter = 0
        
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'train_loss': train_loss,
            'val_loss': val_loss,
            'class_weights': class_weights,
        }, 'retinanet_best.pth')
        
        print(f'New best model saved (improved by {improvement:.4f})')
        writer.add_scalar('Metrics/Best_Val_Loss', best_val_loss, epoch)
    else:
        patience_counter += 1
        print(f'  Patience: {patience_counter}/{PATIENCE}')
        
        if patience_counter >= PATIENCE:
            print(f'\nEarly stopping triggered at epoch {epoch+1}')
            break
    
    # Save last model
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'train_loss': train_loss,
        'val_loss': val_loss,
        'class_weights': class_weights,
    }, 'retinanet_last.pth')
    
    


print(f"Best validation loss: {best_val_loss:.4f}")
print(f"Stopped at epoch: {epoch+1}")




writer.close()

Training set:
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Filtered 571 -> 571 annotations
creating index...
index created!

Validation set:
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Filtered 110 -> 110 annotations
creating index...
index created!

Dataset sizes:
  Train: 488 images
  Val: 93 images

Original class distribution (instances):
  Good: 369 (64.6%)
  Weathered: 172 (30.1%)
  Heavily Damaged: 30 (5.3%)

Class weights (for sampling):
  Good: 0.52
  Weathered: 1.11
  Heavily Damaged: 6.34

Weighted sampler created with 488 samples

Steps per epoch:
  Train: 122 steps
  Val: 24 steps

Using device: cuda

Starting training for 15 epochs


Epoch 1/15 [Train]: 100%|██████████| 122/122 [04:30<00:00,  2.21s/it, loss=0.7219]



Classes seen in epoch 1:
  Good: 184 (32.6%)
  Weathered: 182 (32.2%)
  Heavily Damaged: 199 (35.2%)


Epoch 1/15 [Val]  : 100%|██████████| 24/24 [00:39<00:00,  1.65s/it, loss=0.7639]



Epoch 1/15:
  Train Loss: 0.8389 (cls: 0.6261, bbox: 0.2128)
  Val Loss:   0.7700 (cls: 0.5293, bbox: 0.2407)
  Overfit Ratio: 1.090
  LR: 0.000100
New best model saved (improved by inf)


...


Epoch 11/15:
  Train Loss: 0.2737 (cls: 0.1785, bbox: 0.0952)
  Val Loss:   0.5933 (cls: 0.3659, bbox: 0.2274)
  Overfit Ratio: 0.461
  LR: 0.000050
  Patience: 4/5


Epoch 12/15 [Train]: 100%|██████████| 122/122 [04:39<00:00,  2.29s/it, loss=0.1641]



Classes seen in epoch 12:
  Good: 203 (35.9%)
  Weathered: 195 (34.5%)
  Heavily Damaged: 167 (29.6%)


Epoch 12/15 [Val]  : 100%|██████████| 24/24 [00:39<00:00,  1.66s/it, loss=0.5655]



Epoch 12/15:
  Train Loss: 0.2135 (cls: 0.1325, bbox: 0.0810)
  Val Loss:   0.6049 (cls: 0.3931, bbox: 0.2118)
  Overfit Ratio: 0.353
  LR: 0.000050
  Patience: 5/5

Early stopping triggered at epoch 12
Best validation loss: 0.5911
Stopped at epoch: 12


In [ ]:
import torch
import torchvision
from torchvision.models.detection import retinanet_resnet50_fpn
from torchvision.models.detection import RetinaNet_ResNet50_FPN_Weights
from torch.utils.data import DataLoader
from torchvision.datasets import CocoDetection
import torchvision.transforms as T
import json
from tqdm.auto import tqdm
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from collections import defaultdict
import seaborn as sns
from pycocotools.cocoeval import COCOeval
from pycocotools.coco import COCO
import os



test_json = r'COCO-based_COCO_condition\annotations\test.json'
test_img_dir = r'COCO-based_COCO_condition\images\test'


allowed_conditions = ["Good", "Weathered", "Heavily Damaged"]
num_classes = 4

BATCH_SIZE = 4
NUM_WORKERS = 0

# Model checkpoint to evaluate
MODEL_PATH = 'retinanet_best.pth'  

#threshold
CONFIDENCE_THRESHOLD = 0.5



class FilteredCocoDetection(CocoDetection):
    def __init__(self, root, annFile, allowed_conditions):
        super().__init__(root, annFile)
        self.annFile = annFile
        self.allowed_conditions = allowed_conditions
        self._filter_annotations()
    
    def _filter_annotations(self):
        """Filter annotations to only include allowed conditions"""
        with open(self.annFile, 'r') as f:
            data = json.load(f)
        
        filtered_anns = []
        for ann in data['annotations']:
            if 'attributes' in ann and 'condition' in ann['attributes']:
                condition = ann['attributes']['condition'][0]
                if condition in self.allowed_conditions:
                    ann['category_id'] = self.allowed_conditions.index(condition) + 1
                    filtered_anns.append(ann)
        
        print(f"Filtered {len(data['annotations'])} -> {len(filtered_anns)} annotations")
        
        data['annotations'] = filtered_anns
        data['categories'] = [
            {'id': i+1, 'name': cond} 
            for i, cond in enumerate(self.allowed_conditions)
        ]
        
        self.coco.dataset = data
        self.coco.createIndex()
    
    def __getitem__(self, idx):
        """Override to return proper format for RetinaNet"""
        img, target = super().__getitem__(idx)
        img_id = self.ids[idx]
        
        # Convert image to tensor
        img_tensor = T.ToTensor()(img)
        
        # Convert target to proper format
        boxes = []
        labels = []
        
        for obj in target:
            if 'bbox' in obj and 'category_id' in obj:
                x, y, w, h = obj['bbox']
                boxes.append([x, y, x + w, y + h])
                labels.append(obj['category_id'])
        
        if len(boxes) == 0:
            boxes = torch.zeros((0, 4), dtype=torch.float32)
            labels = torch.zeros((0,), dtype=torch.int64)
        else:
            boxes = torch.as_tensor(boxes, dtype=torch.float32)
            labels = torch.as_tensor(labels, dtype=torch.int64)
        
        target_dict = {
            'boxes': boxes,
            'labels': labels,
            'image_id': torch.tensor([img_id])
        }
        
        return img_tensor, target_dict, img  # Return original PIL image too


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

model = retinanet_resnet50_fpn(weights=None)

# Modify classification head
in_features = 256
num_anchors = model.head.classification_head.num_anchors

from torchvision.models.detection.retinanet import RetinaNetClassificationHead
model.head.classification_head = RetinaNetClassificationHead(
    in_channels=in_features,
    num_anchors=num_anchors,
    num_classes=num_classes
)

# Load checkpoint
checkpoint = torch.load(MODEL_PATH, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)
model.eval()

print(f"Loaded model from {MODEL_PATH}")
print(f"  Epoch: {checkpoint['epoch']}")
print(f"  Val Loss: {checkpoint['val_loss']:.4f}")



print("\nCreating test dataset...")
test_dataset = FilteredCocoDetection(
    root=test_img_dir,
    annFile=test_json,
    allowed_conditions=allowed_conditions
)

print(f"Test set: {len(test_dataset)} images")

def collate_fn(batch):
    images = [item[0] for item in batch]
    targets = [item[1] for item in batch]
    orig_images = [item[2] for item in batch]
    return images, targets, orig_images

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    collate_fn=collate_fn
)


def calculate_iou(box1, box2):
    """Calculate IoU between two boxes [x1, y1, x2, y2]"""
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    
    intersection = max(0, x2 - x1) * max(0, y2 - y1)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union = area1 + area2 - intersection
    
    return intersection / union if union > 0 else 0

def match_predictions_to_targets(pred_boxes, pred_labels, pred_scores, 
                                  target_boxes, target_labels, iou_threshold=0.5):
    """Match predictions to ground truth boxes"""
    matches = []
    used_targets = set()
    
    # Sort predictions by score (highest first)
    sorted_indices = torch.argsort(pred_scores, descending=True)
    
    for idx in sorted_indices:
        pred_box = pred_boxes[idx].cpu().numpy()
        pred_label = pred_labels[idx].item()
        pred_score = pred_scores[idx].item()
        
        best_iou = 0
        best_target_idx = -1
        
        for t_idx, (target_box, target_label) in enumerate(zip(target_boxes, target_labels)):
            if t_idx in used_targets:
                continue
            
            target_box = target_box.cpu().numpy()
            target_label = target_label.item()
            
            if pred_label != target_label:
                continue
            
            iou = calculate_iou(pred_box, target_box)
            
            if iou > best_iou:
                best_iou = iou
                best_target_idx = t_idx
        
        if best_iou >= iou_threshold:
            matches.append({
                'pred_idx': idx.item(),
                'target_idx': best_target_idx,
                'iou': best_iou,
                'label': pred_label,
                'score': pred_score,
                'correct': True
            })
            used_targets.add(best_target_idx)
        else:
            matches.append({
                'pred_idx': idx.item(),
                'target_idx': -1,
                'iou': best_iou,
                'label': pred_label,
                'score': pred_score,
                'correct': False
            })
    
    return matches


all_predictions = []
all_targets = []
per_class_stats = defaultdict(lambda: {'tp': 0, 'fp': 0, 'fn': 0, 'ious': []})

with torch.no_grad():
    for images, targets, orig_images in tqdm(test_loader, desc="Evaluating"):
        images = [img.to(device) for img in images]
        
        # Get predictions
        predictions = model(images)
        
        # Process each image in batch
        for pred, target in zip(predictions, targets):
            # Filter by confidence threshold
            keep = pred['scores'] > CONFIDENCE_THRESHOLD
            pred_boxes = pred['boxes'][keep]
            pred_labels = pred['labels'][keep]
            pred_scores = pred['scores'][keep]
            
            target_boxes = target['boxes']
            target_labels = target['labels']
            
            # Match predictions to targets
            matches = match_predictions_to_targets(
                pred_boxes, pred_labels, pred_scores,
                target_boxes, target_labels
            )
            
            # Store for overall metrics
            all_predictions.append({
                'boxes': pred_boxes.cpu(),
                'labels': pred_labels.cpu(),
                'scores': pred_scores.cpu(),
                'image_id': target['image_id'].item()
            })
            
            all_targets.append({
                'boxes': target_boxes.cpu(),
                'labels': target_labels.cpu(),
                'image_id': target['image_id'].item()
            })
            
            # Update per-class statistics
            for match in matches:
                label = match['label']
                if match['correct']:
                    per_class_stats[label]['tp'] += 1
                    per_class_stats[label]['ious'].append(match['iou'])
                else:
                    per_class_stats[label]['fp'] += 1
            
            # Count false negatives (unmatched targets)
            matched_targets = {m['target_idx'] for m in matches if m['correct']}
            for t_idx, label in enumerate(target_labels):
                if t_idx not in matched_targets:
                    per_class_stats[label.item()]['fn'] += 1



print("EVALUATION RESULTS")

# Overall statistics
total_tp = sum(stats['tp'] for stats in per_class_stats.values())
total_fp = sum(stats['fp'] for stats in per_class_stats.values())
total_fn = sum(stats['fn'] for stats in per_class_stats.values())

overall_precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
overall_recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
overall_f1 = 2 * (overall_precision * overall_recall) / (overall_precision + overall_recall) if (overall_precision + overall_recall) > 0 else 0

print(f"\nOverall Metrics (IoU threshold = 0.5):")
print(f"  Precision: {overall_precision:.4f}")
print(f"  Recall:    {overall_recall:.4f}")
print(f"  F1-Score:  {overall_f1:.4f}")

# Per-class metrics
print(f"\nPer-Class Metrics:")
print("-" * 60)

class_metrics = []
for class_id in sorted(per_class_stats.keys()):
    stats = per_class_stats[class_id]
    class_name = allowed_conditions[class_id - 1]
    
    precision = stats['tp'] / (stats['tp'] + stats['fp']) if (stats['tp'] + stats['fp']) > 0 else 0
    recall = stats['tp'] / (stats['tp'] + stats['fn']) if (stats['tp'] + stats['fn']) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    avg_iou = np.mean(stats['ious']) if stats['ious'] else 0
    
    print(f"{class_name}:")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall:    {recall:.4f}")
    print(f"  F1-Score:  {f1:.4f}")
    print(f"  Avg IoU:   {avg_iou:.4f}")
    print(f"  TP: {stats['tp']}, FP: {stats['fp']}, FN: {stats['fn']}")
    print()
    
    class_metrics.append({
        'class': class_name,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'avg_iou': avg_iou
    })

print("\nConfusion Matrix:")


# Initialize confusion matrix
conf_matrix = np.zeros((num_classes-1, num_classes-1), dtype=int)

for pred, target in zip(all_predictions, all_targets):
    pred_labels = pred['labels'].numpy()
    target_labels = target['labels'].numpy()
    pred_boxes = pred['boxes'].numpy()
    target_boxes = target['boxes'].numpy()
    
    # Match predictions to targets
    for target_idx, target_label in enumerate(target_labels):
        target_box = target_boxes[target_idx]
        best_iou = 0
        best_pred_label = None
        
        for pred_idx, pred_label in enumerate(pred_labels):
            pred_box = pred_boxes[pred_idx]
            iou = calculate_iou(pred_box, target_box)
            
            if iou > best_iou:
                best_iou = iou
                best_pred_label = pred_label
        
        if best_iou >= 0.5 and best_pred_label is not None:
            conf_matrix[target_label-1, best_pred_label-1] += 1
        else:
            # Missed detection - could add to confusion matrix or track separately
            pass

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=allowed_conditions,
            yticklabels=allowed_conditions)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix (IoU > 0.5)')
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=150)


fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for class_id, class_name in enumerate(allowed_conditions, start=1):
    # Collect all predictions and ground truths for this class
    class_preds = []
    class_targets_count = 0
    
    for pred, target in zip(all_predictions, all_targets):
        # Count ground truth boxes
        class_targets_count += (target['labels'] == class_id).sum().item()
        
        # Get predictions for this class
        class_mask = pred['labels'] == class_id
        if class_mask.sum() > 0:
            scores = pred['scores'][class_mask].numpy()
            boxes = pred['boxes'][class_mask].numpy()
            
            # Check if predictions match ground truth
            for score, box in zip(scores, boxes):
                is_correct = False
                for target_box, target_label in zip(target['boxes'], target['labels']):
                    if target_label.item() == class_id:
                        iou = calculate_iou(box, target_box.numpy())
                        if iou >= 0.5:
                            is_correct = True
                            break
                
                class_preds.append({'score': score, 'correct': is_correct})
    
    # Sort by score
    class_preds.sort(key=lambda x: x['score'], reverse=True)
    
    # Calculate precision and recall at different thresholds
    precisions = []
    recalls = []
    
    tp = 0
    fp = 0
    
    for pred in class_preds:
        if pred['correct']:
            tp += 1
        else:
            fp += 1
        
        precision = tp / (tp + fp)
        recall = tp / class_targets_count if class_targets_count > 0 else 0
        
        precisions.append(precision)
        recalls.append(recall)
    
    # Plot
    ax = axes[class_id - 1]
    ax.plot(recalls, precisions, linewidth=2)
    ax.set_xlabel('Recall')
    ax.set_ylabel('Precision')
    ax.set_title(f'P-R Curve: {class_name}')
    ax.grid(True, alpha=0.3)
    ax.set_xlim([0, 1])
    ax.set_ylim([0, 1])
    
    # Calculate AP (area under curve)
    ap = np.trapz(precisions, recalls) if recalls else 0
    ax.text(0.5, 0.05, f'AP = {ap:.3f}', transform=ax.transAxes, 
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.savefig('precision_recall_curves.png', dpi=150)

def visualize_predictions(image, pred, target, idx):
    """Visualize predictions vs ground truth"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
    
    # Convert tensor to numpy
    if isinstance(image, torch.Tensor):
        image = image.permute(1, 2, 0).cpu().numpy()
    else:
        image = np.array(image)
    
    # Ground truth
    ax1.imshow(image)
    ax1.set_title('Ground Truth', fontsize=14, fontweight='bold')
    for box, label in zip(target['boxes'], target['labels']):
        box = box.cpu().numpy()
        x1, y1, x2, y2 = box
        rect = patches.Rectangle((x1, y1), x2-x1, y2-y1, 
                                  linewidth=2, edgecolor='green', facecolor='none')
        ax1.add_patch(rect)
        ax1.text(x1, y1-5, allowed_conditions[label-1], 
                 color='white', fontsize=10, 
                 bbox=dict(boxstyle='round', facecolor='green', alpha=0.7))
    ax1.axis('off')
    
    # Predictions
    ax2.imshow(image)
    ax2.set_title('Predictions', fontsize=14, fontweight='bold')
    keep = pred['scores'] > CONFIDENCE_THRESHOLD
    for box, label, score in zip(pred['boxes'][keep], pred['labels'][keep], pred['scores'][keep]):
        box = box.cpu().numpy()
        x1, y1, x2, y2 = box
        rect = patches.Rectangle((x1, y1), x2-x1, y2-y1, 
                                  linewidth=2, edgecolor='red', facecolor='none')
        ax2.add_patch(rect)
        ax2.text(x1, y1-5, f"{allowed_conditions[label-1]} ({score:.2f})", 
                 color='white', fontsize=10,
                 bbox=dict(boxstyle='round', facecolor='red', alpha=0.7))
    ax2.axis('off')
    
    plt.tight_layout()
    plt.savefig(f'prediction_sample_{idx}.png', dpi=150, bbox_inches='tight')
    plt.close()

# Visualize first 5 images
print("Saving visualization of first 5 test images...")
with torch.no_grad():
    for idx, (images, targets, orig_images) in enumerate(test_loader):
        if idx >= 5:  # Only visualize first 5 batches
            break
        
        images_gpu = [img.to(device) for img in images]
        predictions = model(images_gpu)
        
        for i, (pred, target, orig_img) in enumerate(zip(predictions, targets, orig_images)):
            visualize_predictions(orig_img, pred, target, idx * BATCH_SIZE + i)




print("EVALUATION SUMMARY")
print(f"Model: {MODEL_PATH}")
print(f"Test images: {len(test_dataset)}")
print(f"Confidence threshold: {CONFIDENCE_THRESHOLD}")
print(f"\nOverall Performance:")
print(f"  Precision: {overall_precision:.4f}")
print(f"  Recall:    {overall_recall:.4f}")
print(f"  F1-Score:  {overall_f1:.4f}")
